In [1]:
from helpers import *

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from CustomLogisticRegression import CustomLogisticRegression as CLR
import sklearn.discriminant_analysis as DA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

# Compute discrimination

In [15]:
def computeDiscrimination(X_test, prediction, sensitiveAttr):
    X_test = X_test.assign(prediction = prediction)
    protectedGroup = X_test[X_test[sensitiveAttr] == 1]
    unprotectedGroup = X_test[X_test[sensitiveAttr] == 0]
    proportionOfProtected = protectedGroup['prediction'].sum() / protectedGroup[sensitiveAttr].count()
    proportionOfUnprotected = unprotectedGroup['prediction'].sum() / protectedGroup[sensitiveAttr].count()
    discrim = abs(proportionOfProtected - proportionOfUnprotected)
    return discrim
    

# Run Helper

In [ ]:
def run(X, y, X_test, y_test, SA):
#     kfold = model_selection.KFold(n_splits=5, random_state=7)
    kfold = model_selection.StratifiedKFold(n_splits=5)
    for i, model in enumerate(models):
        cv_result = model_selection.cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

        model.fit(X, y)
        prediction_test = model.predict(X_test)

        acc_score_val = np.mean(cv_result)
        acc_score_test = accuracy_score(y_test, prediction_test)
        discr_score = computeDiscrimination(X_test, prediction_test, SA)
        print ('-'*40)
        print ('val: {0}: {1}'.format(names[i], acc_score_val))
        print ('test: {0}: {1}'.format(names[i], acc_score_test))
        print('{0} discrimination: {1}'.format(names[i], discr_score))
        

# Runner with the data splitting according to the report

In [3]:
def runAdultWithSplitting(X, y, X_test, y_test, SA):
    X_splits = np.array_split(X, 5)
    y_splits = np.array_split(y, 5)
    for x_split, y_split in zip(X_splits, y_splits):
        val_amount = math.floor((1/3) * x_split.shape[0]) # 1/3 of 1 split set
        X_val = x_split[0:val_amount]
        y_val = y_split[0:val_amount]
        X_train = x_split[val_amount:]
        y_train = y_split[val_amount:]
#         splitSize = x_split.shape[0]
#         X_train = x_split[0:splitSize-val_amount]
#         y_train = y_split[0:splitSize-val_amount]
#         X_val = x_split[splitSize-val_amount:]
#         y_val = y_split[splitSize-val_amount:]
        for i, model in enumerate(models):
            model.fit(X_train, y_train)
            prediction_val = model.predict(X_val)
            acc_score_val = np.mean(y_val == prediction_val)
            prediction_test = model.predict(X_test)
            acc_score_test = accuracy_score(y_test, prediction_test)
            discr_score = computeDiscrimination(X_test, prediction_test, SA)
            print ('-'*40)
            print ('val: {0}: {1}'.format(names[i], acc_score_val))
            print ('test: {0}: {1}'.format(names[i], acc_score_test))
            print('{0} discrimination: {1}'.format(names[i], discr_score))
        print ('-'*80)
        print ('-'*80)

In [4]:
def runGermanWithSplitting(X, y, SA):
    X_splits = np.array_split(X, 5)
    y_splits = np.array_split(y, 5)
    for x_split, y_split in zip(X_splits, y_splits):
        train_amount = math.floor(0.5 * x_split.shape[0]) # 50% of 1 split set (round)
        X_train = x_split[0:train_amount]
        y_train = y_split[0:train_amount]
        val_amount = math.floor(0.2 * x_split.shape[0]) # 20% of 1 split set
        X_val = x_split[train_amount:train_amount+val_amount]
        y_val = y_split[train_amount:train_amount+val_amount]
        X_test = x_split[train_amount+val_amount:] # 30% of 1 split set
        y_test = y_split[train_amount+val_amount:]
        for i, model in enumerate(models):
            model.fit(X_train, y_train)
            prediction_val = model.predict(X_val)
            acc_score_val = np.mean(y_val == prediction_val)
            prediction_test = model.predict(X_test)
            acc_score_test = accuracy_score(y_test, prediction_test)
            discr_score = computeDiscrimination(X_test, prediction_test, SA)
            print ('-'*40)
            print ('val: {0}: {1}'.format(names[i], acc_score_val))
            print ('test: {0}: {1}'.format(names[i], acc_score_test))
            print('{0} discrimination: {1}'.format(names[i], discr_score))
        print ('-'*80)
        print ('-'*80)

# Models to Run

In [5]:
models = []
names = ['LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10)',
         'LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10)',
         'LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=100, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=500, penalty=none, C=1e10)',
         'LR(solver=newton-cg, fit_intercept=False, max_iter=1500, penalty=none, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=1000, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=2000, C=1e10)',
         'LR(solver=liblinear, fit_intercept=False, max_iter=3000, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=1000, penalty=none, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10)',
         'LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10)',
         'LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10)',
         'Custom Logistic Regression(lr=0.1, max_itr=100)',
         'Custom Logistic Regression(lr=0.1, max_itr=500)',
         'Custom Logistic Regression(lr=0.1, max_itr=1500)',
         'Custom Logistic Regression(lr=0.01, max_itr=100)',
         'Custom Logistic Regression(lr=0.01, max_itr=500)',
         'Custom Logistic Regression(lr=0.01, max_itr=1500)',
         'Custom Logistic Regression(lr=0.001, max_itr=100)',
         'Custom Logistic Regression(lr=0.001, max_itr=500)',
         'Custom Logistic Regression(lr=0.001, max_itr=1500)',
         'Custom Logistic Regression(lr=0.0001, max_itr=100)',
         'Custom Logistic Regression(lr=0.0001, max_itr=500)',
         'Custom Logistic Regression(lr=0.0001, max_itr=1500)',
#          'Linear Discriminant Analysis',
#          'Quadratic Discriminant Analysis',
#          'Random Forest',
#          'Neural Network',
#          'Gaussian NB',
#          'Bernoulli NB',
#          'Complement NB',
#          'Multinomial NB',
#          'Random Forest',
#          'K Neighbors Classifier',
#          'SVM',
#          'LinearSVC'
]

models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=700, penalty='none'))
models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=1400, penalty='none'))
models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=2100, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=100, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=500, penalty='none'))
models.append(LogisticRegression(solver='newton-cg', fit_intercept=False, max_iter=1500, penalty='none'))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=1000, C=1e10))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=2000, C=1e10))
models.append(LogisticRegression(solver='liblinear', fit_intercept=False, max_iter=3000, C=1e10))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=1000, penalty='none'))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=3000, penalty='none'))
models.append(LogisticRegression(solver='sag', fit_intercept=False, max_iter=5000, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=700, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=1400, penalty='none'))
models.append(LogisticRegression(solver='saga', fit_intercept=False, max_iter=2100, penalty='none'))
models.append(CLR(0.1, 100))
models.append(CLR(0.1, 500))
models.append(CLR(0.1, 1500))
models.append(CLR(0.01, 100))
models.append(CLR(0.01, 500))
models.append(CLR(0.01, 1500))
models.append(CLR(0.001, 100))
models.append(CLR(0.001, 500))
models.append(CLR(0.001, 1500))
models.append(CLR(0.0001, 100))
models.append(CLR(0.0001, 500))
models.append(CLR(0.0001, 1500))
# models.append(DA.LinearDiscriminantAnalysis())
# models.append(DA.QuadraticDiscriminantAnalysis())
# models.append(RandomForestClassifier(n_estimators=100))
# models.append(MLPClassifier())
# models.append(GaussianNB())
# models.append(BernoulliNB())
# models.append(ComplementNB())
# models.append(MultinomialNB())
# models.append(DecisionTreeClassifier())
# models.append(KNeighborsClassifier(n_neighbors=50))
# models.append(SVC())
# models.append(LinearSVC())

# Adult Dataset

In [6]:
df_adult, pct = load_adult('datasets/adult/adult.data')
X_adult = df_adult.iloc[:, :-1]
y_adult = df_adult.iloc[:, -1]
print('percentage of corrupt rows: {0:.1f}%'.format((1-pct)*100))

df_adult_test, pct = load_adult('datasets/adult/adult.test')
X_adult_test = df_adult_test.iloc[:, :-1]
y_adult_test = df_adult_test.iloc[:, -1]
print('percentage of corrupt rows in testing: {0:.1f}%'.format((1-pct)*100))

percentage of corrupt rows: 7.4%
percentage of corrupt rows in testing: 7.5%


# expanding

In [7]:
X_adult_all = X_adult.append(X_adult_test)
X_adult_all_expand = pd.get_dummies(X_adult_all)
X_expand = X_adult_all_expand[0:X_adult.shape[0]]
X_expand_test = X_adult_all_expand[X_adult.shape[0]:]

# LabelEncoder

In [ ]:
# encoders = {"workclass": preprocessing.LabelEncoder(), 
#             "education": preprocessing.LabelEncoder(), 
#             "marital-status": preprocessing.LabelEncoder(), 
#             "occupation": preprocessing.LabelEncoder(), 
#             "relationship": preprocessing.LabelEncoder(), 
#             "race": preprocessing.LabelEncoder(), 
#             "sex": preprocessing.LabelEncoder(), 
#             "native-country": preprocessing.LabelEncoder()}

# X_encoded = encode(X_adult, encoders)
# X_encoded_test = encode(X_adult_test, encoders)

In [8]:
# run(X_expand, y_adult, X_expand_test, y_adult_test, 'sex')
runAdultWithSplitting(X_expand, y_adult, X_expand_test, y_adult_test, 'sex')

284
4913
1097
4913
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.7876678269517653
test: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.7944887118193891
LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 0.1654793405251374
284
4913
1097
4913
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.7876678269517653
test: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.7944887118193891
LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 0.1654793405251374
284
4913
1097
4913
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.7876678269517653
test: LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.7944887118193891


/usr/local/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


421
4913
2645
4913
----------------------------------------
val: LR(solver=newton-cg, fit_intercept=False, max_iter=100, penalty=none, C=1e10): 0.8398806563898558
test: LR(solver=newton-cg, fit_intercept=False, max_iter=100, penalty=none, C=1e10): 0.8443559096945551
LR(solver=newton-cg, fit_intercept=False, max_iter=100, penalty=none, C=1e10) discrimination: 0.4526765723590474


/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

446
4913
2708
4913
----------------------------------------
val: LR(solver=newton-cg, fit_intercept=False, max_iter=500, penalty=none, C=1e10): 0.8368970661362506
test: LR(solver=newton-cg, fit_intercept=False, max_iter=500, penalty=none, C=1e10): 0.8423638778220451
LR(solver=newton-cg, fit_intercept=False, max_iter=500, penalty=none, C=1e10) discrimination: 0.46041115408100963


...

210
4913
812
4913
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.7880597014925373
test: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.7859229747675963
LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 0.1225320578058213


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


225
4913
852
4913
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.7875621890547264
test: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.7854581673306773
LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 0.12762059841237533


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antongladyr/McGill/Applied Machine Learning/learning-fair-representations/CustomLogisticRegression.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-a))


238
4913
891
4913
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.7855721393034826
test: LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.7854581673306773
LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10) discrimination: 0.13291268064319153
2.0
4913
22.0
4913
----------------------------------------
val: Custom Logistic Regression(lr=0.1, max_itr=100): 0.755223880597015
test: Custom Logistic Regression(lr=0.1, max_itr=100): 0.7559096945551129
Custom Logistic Regression(lr=0.1, max_itr=100) discrimination: 0.004070832485243232
12.0
4913
79.0
4913
----------------------------------------
val: Custom Logistic Regression(lr=0.1, max_itr=500): 0.7597014925373134
test: Custom Logistic Regression(lr=0.1, max_itr=500): 0.7603585657370517
Custom Logistic Regression(lr=0.1, max_itr=500) discrimination: 0.013637288825564827
64.0
4913
275.0
4913
---------------------------

# German Dataset

In [9]:
# load german dataset
df_german = load_german('datasets/german/german.data')
X_german = df_german.iloc[:, :-1]
y_german = df_german.iloc[:, -1]

X_german.loc[X_german['Age'] <= 25, 'Age'] = 1
X_german.loc[X_german['Age'] > 25, 'Age'] = 0

# One hot encoder
X_german_encoded = pd.get_dummies(X_german)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_german_encoded, y_german, test_size=0.3, shuffle=False)
# X_train, X_test, y_train, y_test = train_test_split(X_german, y_german, test_size=0.3, random_state=42)


In [10]:
# run(X_train, y_train, X_test, y_test, 'Age')
runGermanWithSplitting(X_german_encoded, y_german, 'Age');

2
8
10
8
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.675
test: LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.65
LR(solver=lbfgs, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 1.0
2
8
10
8
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.675
test: LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.65
LR(solver=lbfgs, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 1.0
2
8
10
8
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.675
test: LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.65
LR(solver=lbfgs, fit_intercept=False, max_iter=2100, penalty=none, C=1e10) discrimination: 1.0
2
8
12
8
-------------------------------------

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0
8
0
8
----------------------------------------
val: LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10): 0.725
test: LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10): 0.65
LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10) discrimination: 0.0


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0
8
0
8
----------------------------------------
val: LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10): 0.725
test: LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10): 0.65
LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10) discrimination: 0.0
0
8
0
8
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.725
test: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.65
LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 0.0
0
8
0
8
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.725
test: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.65
LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 0.0


...

0
8
0
8
----------------------------------------
val: LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10): 0.625
test: LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10): 0.7166666666666667
LR(solver=sag, fit_intercept=False, max_iter=3000, penalty=none, C=1e10) discrimination: 0.0


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0
8
0
8
----------------------------------------
val: LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10): 0.625
test: LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10): 0.7166666666666667
LR(solver=sag, fit_intercept=False, max_iter=5000, penalty=none, C=1e10) discrimination: 0.0
0
8
0
8
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.625
test: LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10): 0.7166666666666667
LR(solver=saga, fit_intercept=False, max_iter=700, penalty=none, C=1e10) discrimination: 0.0
0
8
0
8
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.625
test: LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10): 0.7166666666666667
LR(solver=saga, fit_intercept=False, max_iter=1400, penalty=none, C=1e10) discrimination: 0.0


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antongladyr/McGill/Applied Machine Learning/learning-fair-representations/CustomLogisticRegression.py:14: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-a))


0
8
0
8
----------------------------------------
val: LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.625
test: LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10): 0.7166666666666667
LR(solver=saga, fit_intercept=False, max_iter=2100, penalty=none, C=1e10) discrimination: 0.0
8.0
8
52.0
8
----------------------------------------
val: Custom Logistic Regression(lr=0.1, max_itr=100): 0.375
test: Custom Logistic Regression(lr=0.1, max_itr=100): 0.2833333333333333
Custom Logistic Regression(lr=0.1, max_itr=100) discrimination: 5.5
0.0
8
0.0
8
----------------------------------------
val: Custom Logistic Regression(lr=0.1, max_itr=500): 0.625
test: Custom Logistic Regression(lr=0.1, max_itr=500): 0.7166666666666667
Custom Logistic Regression(lr=0.1, max_itr=500) discrimination: 0.0
0.0
8
0.0
8
----------------------------------------
val: Custom Logistic Regression(lr=0.1, max_itr=1500): 0.625
test: Custom Logistic Regression(lr=0.1, max_i

# The most similar model

In [47]:
models = []
names = ['LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10)',
]

models.append(LogisticRegression(solver='lbfgs', fit_intercept=False, max_iter=10000, penalty='none'))

In [48]:
runAdultWithSplitting(X_expand, y_adult, X_expand_test, y_adult_test, 'sex')

----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.7876678269517653
test: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.7944887118193891
LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10) discrimination: 0.1654793405251374
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.7951268025857782
test: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.7935590969455512
LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10) discrimination: 0.16792184001628335
--------------------------------------------------------------------------------
----------------------------------------------

In [49]:
runGermanWithSplitting(X_german_encoded, y_german, 'Age');

----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.675
test: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.65
LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10) discrimination: 1.0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
----------------------------------------
val: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.6
test: LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10): 0.7
LR(solver=lbfgs, fit_intercept=False, max_iter=10000, penalty=none, C=1e10) discrimination: 0.6000000000000001
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------